In [100]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
load_dotenv()

True

# OPM Employment Data
FedScope Employment Cube (March 2024)  
FACTDATA_MAR2024.TXT  
from  
https://www.opm.gov/data/datasets/  

(will require download)

In [101]:
file_path = "../datasets/OPM/FedScope Employment Cube (March 2024)/FACTDATA_MAR2024.TXT"
fed_employment = pd.read_csv(file_path, sep=",", engine="python", on_bad_lines="skip", encoding="utf-8")
fed_employment.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,20,XXXX,2,50,F,1,202403,1,210000.0,20.8
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,20,XXXX,2,50,F,1,202403,1,203000.0,31.2
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,30,XXXX,2,48,F,2,202403,1,NaN,22.0
3,AA00,11,C,04,12,B,0560,2,99,GS-12,9,XXXX,8,15,F,1,202403,1,99200.0,2.5
4,AA00,11,D,04,13,B,0905,1,99,GS-13,11,XXXX,8,30,F,1,202403,1,117962.0,2.5


# OPM Agency by Code
FedScope Employment Cube (March 2024) 
DTagy.txt  
from  
https://www.opm.gov/data/datasets/

In [102]:
# Extract those codes from a messy text file.
# https://acf.gov/sites/default/files/documents/paris/federal_codes_for_states_03212018.pdf

file_path = "../datasets/OPM/FedScope Employment Cube (March 2024)/DTagy.txt"
agency_codes = pd.read_csv(file_path, sep=",", engine="python", on_bad_lines="skip", encoding="utf-8")

def clean_agency_name(agency_name):
    name = "-".join(agency_name.split("-")[1:])
    name = name.strip()
    name = " ".join(name.split())
    return name

agency_codes['AGYT'] = agency_codes.apply(lambda row: clean_agency_name(row['AGYT']), axis=1)

agency_mappings = {
    "DEPARTMENT OF THE ARMY": "DEPARTMENT OF DEFENSE",
    "DEPARTMENT OF THE NAVY": "DEPARTMENT OF DEFENSE",
    "DEPARTMENT OF THE AIR FORCE": "DEPARTMENT OF DEFENSE",
    "U.S. AGENCY FOR INTERNATIONAL DEVELOPMENT": "AGENCY FOR INTERNATIONAL DEVELOPMENT",
    
}

def applymap(agyname):
    if agyname in agency_mappings:
        return agency_mappings[agyname]
    else: return agyname

agency_codes['AGYT'] = agency_codes.apply(lambda row: applymap(row['AGYT']), axis=1)
agency_codes

,AGYTYP,AGYTYPT,AGY,AGYT,AGYSUB,AGYSUBT
0,1,Cabinet Level Agencies,AF,DEPARTMENT OF DEFENSE,AF02,AF02-AIR FORCE INSPECTION AGENCY (FO)
1,1,Cabinet Level Agencies,AF,DEPARTMENT OF DEFENSE,AF03,AF03-AIR FORCE OPERATIONAL TEST AND EVALUATION...
2,1,Cabinet Level Agencies,AF,DEPARTMENT OF DEFENSE,AF06,AF06-AIR FORCE AUDIT AGENCY
3,1,Cabinet Level Agencies,AF,DEPARTMENT OF DEFENSE,AF07,AF07-AIR FORCE OFFICE OF SPECIAL INVESTIGATIONS
4,1,Cabinet Level Agencies,AF,DEPARTMENT OF DEFENSE,AF09,AF09-AIR FORCE PERSONNEL CENTER
...,...,...,...,...,...,...
537,4,Small Independent Agencies (less than 100 empl...,WK,FEDERAL PERMITTING IMPROVEMENT STEERING COUNCIL,WK00,WK00-FEDERAL PERMITTING IMPROVEMENT STEERING C...
538,4,Small Independent Agencies (less than 100 empl...,WU,SOUTHWEST BORDER REGIONAL COMMISSION,WU00,WU00-SOUTHWEST BORDER REGIONAL COMMISSION
539,4,Small Independent Agencies (less than 100 empl...,WX,WORLD WAR 1 CENTENNIAL COMMISSION,WX00,WX00-WORLD WAR 1 CENTENNIAL COMMISSION
540,4,Small Independent Agencies (less than 100 empl...,ZP,U.S. COMMISSION ON INTERNATIONAL RELIGIOUS FRE...,ZP00,ZP00-U.S. COMMISSION ON INTERNATIONAL RELIGIOU...


In [103]:
agency_codes['AGYTYPT'].unique()

array(['Cabinet Level Agencies',
       'Large Independent Agencies (1000 or more employees)',
       'Medium Independent Agencies (100-999 employees)',
       'Small Independent Agencies (less than 100 employees)'],
      dtype=object)

In [104]:
agency_codes[agency_codes['AGYTYPT'].isin(["Large Independent Agencies (1000 or more employees)", "Medium Independent Agencies (100-999 employees)", "Small Independent Agencies (less than 100 employees)"])]

,AGYTYP,AGYTYPT,AGY,AGYT,AGYSUB,AGYSUBT
394,2,Large Independent Agencies (1000 or more emplo...,AM,AGENCY FOR INTERNATIONAL DEVELOPMENT,AM00,AM00-U.S. AGENCY FOR INTERNATIONAL DEVELOPMENT
395,2,Large Independent Agencies (1000 or more emplo...,CU,NATIONAL CREDIT UNION ADMINISTRATION,CU00,CU00-NATIONAL CREDIT UNION ADMINISTRATION
396,2,Large Independent Agencies (1000 or more emplo...,EE,EQUAL EMPLOYMENT OPPORTUNITY COMMISSION,EE00,EE00-EQUAL EMPLOYMENT OPPORTUNITY COMMISSION
397,2,Large Independent Agencies (1000 or more emplo...,EP,ENVIRONMENTAL PROTECTION AGENCY,EP00,EP00-ENVIRONMENTAL PROTECTION AGENCY
398,2,Large Independent Agencies (1000 or more emplo...,FC,FEDERAL COMMUNICATIONS COMMISSION,FC00,FC00-FEDERAL COMMUNICATIONS COMMISSION
...,...,...,...,...,...,...
537,4,Small Independent Agencies (less than 100 empl...,WK,FEDERAL PERMITTING IMPROVEMENT STEERING COUNCIL,WK00,WK00-FEDERAL PERMITTING IMPROVEMENT STEERING C...
538,4,Small Independent Agencies (less than 100 empl...,WU,SOUTHWEST BORDER REGIONAL COMMISSION,WU00,WU00-SOUTHWEST BORDER REGIONAL COMMISSION
539,4,Small Independent Agencies (less than 100 empl...,WX,WORLD WAR 1 CENTENNIAL COMMISSION,WX00,WX00-WORLD WAR 1 CENTENNIAL COMMISSION
540,4,Small Independent Agencies (less than 100 empl...,ZP,U.S. COMMISSION ON INTERNATIONAL RELIGIOUS FRE...,ZP00,ZP00-U.S. COMMISSION ON INTERNATIONAL RELIGIOU...


# Budget by Agency

In [105]:
with open("../datasets/OPM/Misc/budget_by_agency.html", 'r') as f:
    raw_table = f.read()

budgets = pd.read_html(raw_table)[0]

### Light cleaning ###
budgets.rename(columns={
    "Agency Name  Sort table by ascending Agency Name  Sort table by descending Agency Name": "Agency",
    "Budgetary Resources  Sort table by ascending Budgetary Resources  Sort table by descending Budgetary Resources": "Budget",
    "Percent of Total  Sort table by ascending Percent of Total  Sort table by descending Percent of Total": "Percent of Total Budget"
}, inplace=True)

def remove_abbreviation(agency_name):
    name = agency_name.split('(')[0]
    name = name.strip()
    name = " ".join(name.split())
    return name


budgets['Agency'] = budgets.apply(lambda row: remove_abbreviation(row['Agency']), axis=1)
budgets['Agency'] = budgets.apply(lambda row: row['Agency'].upper(), axis=1)
# If math needed on dollar values, uncomment.
# def clean_money(dollars: str) -> str:
#     dollars = dollars.replace('\r', '')
#     dollars = dollars.replace('$', '')
#     dollars = dollars.replace(',', '')
#     return float(dollars)
    

# budgets['Budget'] = budgets.apply(lambda row: clean_money(row['Budget']), axis=1)

budgets.head()
budgets.to_excel("budgets.xlsx")

C:\Users\Aaron\AppData\Local\Temp\ipykernel_92636\1784220426.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  budgets = pd.read_html(raw_table)[0]


In [106]:



agy_1 = list(agency_codes['AGYT'].unique())
agy_2 = list(budgets['Agency'].unique())

intersection = [item for item in agy_1 if item in agy_2]
needs_correction = [item for item in agy_2 if item not in agy_1]

options = [item for item in agy_1 if item not in intersection]

print("\n".join(agy_1))

DEPARTMENT OF DEFENSE
DEPARTMENT OF AGRICULTURE
DEPARTMENT OF COMMERCE
DEPARTMENT OF JUSTICE
DEPARTMENT OF LABOR
DEPARTMENT OF ENERGY
DEPARTMENT OF EDUCATION
DEPARTMENT OF HEALTH AND HUMAN SERVICES
DEPARTMENT OF HOMELAND SECURITY
DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT
DEPARTMENT OF THE INTERIOR
DEPARTMENT OF STATE
DEPARTMENT OF TRANSPORTATION
DEPARTMENT OF THE TREASURY
DEPARTMENT OF VETERANS AFFAIRS
AGENCY FOR INTERNATIONAL DEVELOPMENT
NATIONAL CREDIT UNION ADMINISTRATION
EQUAL EMPLOYMENT OPPORTUNITY COMMISSION
ENVIRONMENTAL PROTECTION AGENCY
FEDERAL COMMUNICATIONS COMMISSION
FEDERAL DEPOSIT INSURANCE CORPORATION
COURT SERVICES AND OFFENDER SUPERVISION AGENCY FOR THE DISTRICT OF COLUMBIA
FEDERAL RESERVE SYSTEM
FEDERAL TRADE COMMISSION
GENERAL SERVICES ADMINISTRATION
U.S. AGENCY FOR GLOBAL MEDIA
GOVERNMENT PRINTING OFFICE
NATIONAL SCIENCE FOUNDATION
NATIONAL LABOR RELATIONS BOARD
NATIONAL AERONAUTICS AND SPACE ADMINISTRATION
NATIONAL ARCHIVES AND RECORDS ADMINISTRATION
NUCLEAR REGU

# Combination of Data

We want to see the number of employees, and budget for each department.  

Step 1: Join the employment data with the agency data to get the names of the agencies and departments. 
Step 3: Group by Department to gather the number of employees per department.  
Step 4: Join the Budget by department table, to see budget for each agency.

In [107]:
# Step 1
fed_employment = fed_employment.merge(agency_codes, on='AGYSUB', how='left')
fed_employment.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,AGYTYP,AGYTYPT,AGY,AGYT,AGYSUBT
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,...,1,202403,1,210000.0,20.8,4,Small Independent Agencies (less than 100 empl...,AA,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,...,1,202403,1,203000.0,31.2,4,Small Independent Agencies (less than 100 empl...,AA,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,...,2,202403,1,NaN,22.0,4,Small Independent Agencies (less than 100 empl...,AA,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...
3,AA00,11,C,04,12,B,0560,2,99,GS-12,...,1,202403,1,99200.0,2.5,4,Small Independent Agencies (less than 100 empl...,AA,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...
4,AA00,11,D,04,13,B,0905,1,99,GS-13,...,1,202403,1,117962.0,2.5,4,Small Independent Agencies (less than 100 empl...,AA,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...


In [108]:
# Step 3
employment_summary_dept = fed_employment[['AGYSUB', 'AGYT']]
employment_summary_dept = employment_summary_dept.groupby('AGYT').size().reset_index(name='Number of Employees')
employment_summary_dept

,AGYT,Number of Employees
0,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,11
1,ADVISORY COUNCIL ON HISTORIC PRESERVATION,53
2,AFRICAN DEVELOPMENT FOUNDATION,30
3,AGENCY FOR INTERNATIONAL DEVELOPMENT,4675
4,AMERICAN BATTLE MONUMENTS COMMISSION,74
...,...,...
124,U.S. INTERAGENCY COUNCIL ON HOMELESSNESS,16
125,U.S. INTERNATIONAL TRADE COMMISSION,414
126,UNITED STATES-CHINA ECONOMIC AND SECURITY REVI...,28
127,UTAH RECLAMATION MITIGATION AND CONSERVATION C...,14


In [109]:
# Step 4
employment_summary_dept = employment_summary_dept.merge(budgets, left_on='AGYT', right_on='Agency', how='outer')
employment_summary_dept

,AGYT,Number of Employees,Agency,Congressional Justification of Budget (CJ),Budget,Percent of Total Budget
0,NaN,NaN,ACCESS BOARD,https://www.access-board.gov/cj,"$7,998,645",Less than 0.01%
1,NaN,NaN,ADMINISTRATIVE CONFERENCE OF THE U.S.,https://www.acus.gov/cj,"$1,685,674",Less than 0.01%
2,ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,11.0,NaN,NaN,NaN,NaN
3,ADVISORY COUNCIL ON HISTORIC PRESERVATION,53.0,ADVISORY COUNCIL ON HISTORIC PRESERVATION,https://www.achp.gov/sites/default/files/2021-...,"$10,714,903",Less than 0.01%
4,AFRICAN DEVELOPMENT FOUNDATION,30.0,AFRICAN DEVELOPMENT FOUNDATION,https://www.usadf.gov/cj,"$38,953,733",Less than 0.01%
...,...,...,...,...,...,...
146,NaN,NaN,UNITED STATES TRADE AND DEVELOPMENT AGENCY,https://www.ustda.gov/cj,"$127,522,040",Less than 0.01%
147,UNITED STATES-CHINA ECONOMIC AND SECURITY REVI...,28.0,NaN,NaN,NaN,NaN
148,UTAH RECLAMATION MITIGATION AND CONSERVATION C...,14.0,NaN,NaN,NaN,NaN
149,NaN,NaN,VIETNAM EDUCATION FOUNDATION,not available,$0,Less than 0.01%


In [110]:
employment_summary_dept.sort_values(by='Number of Employees', ascending=False).to_csv("../data summaries/agencies_budget_employees_summary.csv")